In [2]:
# 导包
from IPython.display import Markdown as md
from matplotlib import pyplot as plt
import sqlite3
import pandas as pd
import numpy as np
import json 
from copy import deepcopy 
from datetime import datetime

In [3]:
df=pd.read_csv(r'tmall_order_report.csv')
#将nan值换成null
df=df.where(pd.notnull(df),None)
#创建一个新的列【订单创建日期
df['订单创建日期']=df['订单创建时间'].apply(lambda x:pd.Timestamp(x).date()).astype(str)
# 将数据写入一张名为example_table的表
with sqlite3.connect('example.db') as conn:
    df.to_sql('example_table',con=conn,if_exists='replace',index=False)
connection=sqlite3.connect('example.db')

In [4]:
df.columns

Index(['订单编号', '总金额', '买家实际支付金额', '收货地址 ', '订单创建时间', '订单付款时间 ', '退款金额',
       '订单创建日期'],
      dtype='object')

In [5]:
df.head()

,订单编号,总金额,买家实际支付金额,收货地址,订单创建时间,订单付款时间,退款金额,订单创建日期
0,1,178.8,0.0,上海,2020-02-21 00:00:00,None,0.0,2020-02-21
1,2,21.0,21.0,内蒙古自治区,2020-02-20 23:59:54,2020-02-21 00:00:02,0.0,2020-02-20
2,3,37.0,0.0,安徽省,2020-02-20 23:59:35,None,0.0,2020-02-20
3,4,157.0,157.0,湖南省,2020-02-20 23:58:34,2020-02-20 23:58:44,0.0,2020-02-20
4,5,64.8,0.0,江苏省,2020-02-20 23:57:04,2020-02-20 23:57:11,64.8,2020-02-20


In [6]:
# 上界为当前行向上取3行，下界不设限制的表达式
# between  unbounded  precending  and 3 preceding


# rows between  current row and 1 following
# 取当前行和在表中位于当前行下一行的数据

# 基于位置关系，可以用rows来代表这个规则


# frame数据与轴之间的关系是基于某个字段的具体值的大小关系，可以用关键字range来代表这个规则

# range between 2 preceding  and current row

# ORDER BY price, stock_number ROWS BETWEEN UNBOUNDED PRECEDING AND 1 FOLLOWING 
# 表示为  先将price，stock_number 数据排序，向下取1行


# PARTITION BY price ORDER BY stock_number rows between unbounded  preceding  and 1 following
# 根据价格进行划分，按照stock_number进行排序，取从第一行到当前行以及下一行的数据


In [7]:
query=""" 
select
    t1.order_date,
    sum(orders_num)
    over(                   ---over 关键字，内定frame
        order by order_date
        rows                ---表当中的相对位置
        between             ---界定上下界
            unbounded preceding  -- 上界，不设限制
        and
            current row --下界，当前行
    )as running_total
from
(   
    select 
        "订单创建日期" as order_date,
        count("订单编号") as orders_num
    from example_table
    group by "订单创建日期"
) as t1 ----按照日期生成当日订单量
"""
result=connection.execute(query).fetchall()
result

[('2020-02-01', 176),
 ('2020-02-02', 398),
 ('2020-02-03', 665),
 ('2020-02-04', 1134),
 ('2020-02-05', 1503),
 ('2020-02-06', 1647),
 ('2020-02-07', 1824),
 ('2020-02-09', 2228),
 ('2020-02-10', 2255),
 ('2020-02-11', 2270),
 ('2020-02-12', 2271),
 ('2020-02-13', 2276),
 ('2020-02-14', 2283),
 ('2020-02-15', 2288),
 ('2020-02-17', 2678),
 ('2020-02-18', 3693),
 ('2020-02-19', 4718),
 ('2020-02-20', 6063),
 ('2020-02-21', 8131),
 ('2020-02-22', 10158),
 ('2020-02-23', 12358),
 ('2020-02-24', 14356),
 ('2020-02-25', 17772),
 ('2020-02-26', 20621),
 ('2020-02-27', 23207),
 ('2020-02-28', 25898),
 ('2020-02-29', 28010)]

In [8]:
query=''' 
select 
    t1.order_date,
    t1.address,
    sum(orders_num),
    over(
        partition by address
        order by order_date
        rows 
        between
            unbounded preceding
        and
            current row
    )as running_total
from 
    (
        select 
            "订单创建日期" as order_date,
            "收获地址" as address,
            count("订单编号") as orders_num
        from 
        example_table
        group by "订单创建日期","收获地址"
    )as t1 
where t1.address IN('北京','上海')

'''
result=connection.execute(query).fetchall()
result

OperationalError: near "by": syntax error

In [ ]:
query = '''
SELECT
      t1.order_date
      ,t1.address
      ,SUM(orders_num)                  -- 聚合函数 SUM()
       OVER(                            -- OVER 关键字，（）内定 frame
             PARTITION BY address       -- 每个 frame 内只有 address 相同的数据
             ORDER BY order_date        -- 按照 order_date 排序 
             ROWS                       -- 按照表当中的相对位置
             BETWEEN                    -- 界定上下界
                  UNBOUNDED PRECEDING   -- 上界，不设限制
             AND 
                  CURRENT ROW           -- 下界，当前行
             ) AS running_total

FROM
      (SELECT "订单创建日期" AS order_date
               ,"收货地址 " AS address
               ,COUNT("订单编号") AS orders_num
      FROM example_table
      GROUP BY "订单创建日期","收货地址 "
      )AS t1                      -- 按照日期，收获地址生成当日订单量

WHERE t1.address IN ('北京','上海')
'''
result = connection.execute(query).fetchall()
result

[('2020-02-01', '上海', 48),
 ('2020-02-02', '上海', 114),
 ('2020-02-03', '上海', 197),
 ('2020-02-04', '上海', 330),
 ('2020-02-05', '上海', 414),
 ('2020-02-06', '上海', 459),
 ('2020-02-07', '上海', 499),
 ('2020-02-09', '上海', 596),
 ('2020-02-10', '上海', 603),
 ('2020-02-11', '上海', 608),
 ('2020-02-13', '上海', 609),
 ('2020-02-14', '上海', 612),
 ('2020-02-17', '上海', 685),
 ('2020-02-18', '上海', 834),
 ('2020-02-19', '上海', 963),
 ('2020-02-20', '上海', 1090),
 ('2020-02-21', '上海', 1291),
 ('2020-02-22', '上海', 1508),
 ('2020-02-23', '上海', 1717),
 ('2020-02-24', '上海', 1957),
 ('2020-02-25', '上海', 2260),
 ('2020-02-26', '上海', 2571),
 ('2020-02-27', '上海', 2862),
 ('2020-02-28', '上海', 3148),
 ('2020-02-29', '上海', 3353),
 ('2020-02-01', '北京', 10),
 ('2020-02-02', '北京', 23),
 ('2020-02-03', '北京', 36),
 ('2020-02-04', '北京', 59),
 ('2020-02-05', '北京', 94),
 ('2020-02-06', '北京', 98),
 ('2020-02-07', '北京', 108),
 ('2020-02-09', '北京', 147),
 ('2020-02-10', '北京', 151),
 ('2020-02-11', '北京', 153),
 ('2020-02-17', '

In [ ]:
query = '''
SELECT
      t1.order_date
      ,t1.address
      ,SUM(orders_num)                  -- 聚合函数 SUM()
       FILTER(
             WHERE t1.address ='北京')   -- 只保留 address = ‘北京’
       OVER(                            -- OVER 关键字，（）内定 frame
             PARTITION BY address       -- 每个 frame 内只有 address 相同的数据
             ORDER BY order_date        -- 按照 order_date 排序 
             ROWS                       -- 按照表当中的相对位置
             BETWEEN                    -- 界定上下界
                  UNBOUNDED PRECEDING   -- 上界，不设限制
             AND 
                  CURRENT ROW           -- 下界，当前行
             ) AS running_total

FROM
      (SELECT "订单创建日期" AS order_date
               ,"收货地址 " AS address
               ,COUNT("订单编号") AS orders_num
      FROM example_table
      GROUP BY "订单创建日期","收货地址 "
      )AS t1                      -- 按照日期，收获地址生成当日订单量

WHERE t1.address IN ('北京','上海')
'''
result = connection.execute(query).fetchall()
result

[('2020-02-01', '上海', None),
 ('2020-02-02', '上海', None),
 ('2020-02-03', '上海', None),
 ('2020-02-04', '上海', None),
 ('2020-02-05', '上海', None),
 ('2020-02-06', '上海', None),
 ('2020-02-07', '上海', None),
 ('2020-02-09', '上海', None),
 ('2020-02-10', '上海', None),
 ('2020-02-11', '上海', None),
 ('2020-02-13', '上海', None),
 ('2020-02-14', '上海', None),
 ('2020-02-17', '上海', None),
 ('2020-02-18', '上海', None),
 ('2020-02-19', '上海', None),
 ('2020-02-20', '上海', None),
 ('2020-02-21', '上海', None),
 ('2020-02-22', '上海', None),
 ('2020-02-23', '上海', None),
 ('2020-02-24', '上海', None),
 ('2020-02-25', '上海', None),
 ('2020-02-26', '上海', None),
 ('2020-02-27', '上海', None),
 ('2020-02-28', '上海', None),
 ('2020-02-29', '上海', None),
 ('2020-02-01', '北京', 10),
 ('2020-02-02', '北京', 23),
 ('2020-02-03', '北京', 36),
 ('2020-02-04', '北京', 59),
 ('2020-02-05', '北京', 94),
 ('2020-02-06', '北京', 98),
 ('2020-02-07', '北京', 108),
 ('2020-02-09', '北京', 147),
 ('2020-02-10', '北京', 151),
 ('2020-02-11', '北京', 153),
 

In [ ]:
query=""" 
select t1.order_date,
t1.address,
avg(total_payment) 
OVER(PARTITION BY address ORDER BY order_date ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING) AS three_day_avg
from
(select
"订单创建日期" as order_date,
"收货地址" as address,
sum("总金额") as total_payment
from example_table
group by "订单创建日期","收货地址") as t1



"""
result = connection.execute(query).fetchall()
result

[('2020-02-01', '收货地址', 20296.0),
 ('2020-02-02', '收货地址', 20983.0),
 ('2020-02-03', '收货地址', 29039.296666666676),
 ('2020-02-04', '收货地址', 32134.63000000001),
 ('2020-02-05', '收货地址', 29405.63000000001),
 ('2020-02-06', '收货地址', 20653.333333333332),
 ('2020-02-07', '收货地址', 22462.333333333332),
 ('2020-02-09', '收货地址', 19195.333333333332),
 ('2020-02-10', '收货地址', 14462.333333333334),
 ('2020-02-11', '收货地址', 1592.3333333333333),
 ('2020-02-12', '收货地址', 172.66666666666666),
 ('2020-02-13', '收货地址', 171.66666666666666),
 ('2020-02-14', '收货地址', 198.66666666666666),
 ('2020-02-15', '收货地址', 11076.233333333343),
 ('2020-02-17', '收货地址', 42692.076666666566),
 ('2020-02-18', '收货地址', 72132.14333333325),
 ('2020-02-19', '收货地址', 108308.50999999978),
 ('2020-02-20', '收货地址', 147913.0333333328),
 ('2020-02-21', '收货地址', 183469.4333333325),
 ('2020-02-22', '收货地址', 207882.99999999892),
 ('2020-02-23', '收货地址', 269380.96666666615),
 ('2020-02-24', '收货地址', 317212.2999999999),
 ('2020-02-25', '收货地址', 346576.3333333

In [ ]:
query=""" 
SELECT 
    t1.order_date,
    t1.address,
    AVG(total_payment) OVER (
        PARTITION BY address 
        ORDER BY order_date 
        ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING
    ) AS three_day_avg
FROM (
    SELECT 
        "订单创建日期" AS order_date,
        "收货地址" AS address,
        SUM("总金额") AS total_payment
    FROM example_table
    GROUP BY "订单创建日期", "收货地址"
) AS t1;


"""
result = connection.execute(query).fetchall()
result

In [ ]:
query=""" 

select 
    t1.order_date,
    t1.address,
    avg(total_payment) over(
        partition by address 
        order by order_date 
        rows between 1 PRECEDING and 1 following) as three_day_avg
from(
    select "订单创建日期" as order_date,
    "收货地址" as address,
    sum("总金额") as total_payment
    from example_table
    group by "订单创建日期","收货地址" )as t1


"""
result = connection.execute(query).fetchall()
result

[('2020-02-01', '收货地址', 20296.0),
 ('2020-02-02', '收货地址', 20983.0),
 ('2020-02-03', '收货地址', 29039.296666666676),
 ('2020-02-04', '收货地址', 32134.63000000001),
 ('2020-02-05', '收货地址', 29405.63000000001),
 ('2020-02-06', '收货地址', 20653.333333333332),
 ('2020-02-07', '收货地址', 22462.333333333332),
 ('2020-02-09', '收货地址', 19195.333333333332),
 ('2020-02-10', '收货地址', 14462.333333333334),
 ('2020-02-11', '收货地址', 1592.3333333333333),
 ('2020-02-12', '收货地址', 172.66666666666666),
 ('2020-02-13', '收货地址', 171.66666666666666),
 ('2020-02-14', '收货地址', 198.66666666666666),
 ('2020-02-15', '收货地址', 11076.233333333343),
 ('2020-02-17', '收货地址', 42692.076666666566),
 ('2020-02-18', '收货地址', 72132.14333333325),
 ('2020-02-19', '收货地址', 108308.50999999978),
 ('2020-02-20', '收货地址', 147913.0333333328),
 ('2020-02-21', '收货地址', 183469.4333333325),
 ('2020-02-22', '收货地址', 207882.99999999892),
 ('2020-02-23', '收货地址', 269380.96666666615),
 ('2020-02-24', '收货地址', 317212.2999999999),
 ('2020-02-25', '收货地址', 346576.3333333

In [ ]:
query = '''

SELECT t1.order_date
       ,t1.address
       ,avg(total_payment) OVER(PARTITION BY address ORDER BY order_date ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING) as three_day_avg
FROM    (SELECT "订单创建日期" AS order_date
        ,"收货地址 " AS address
        ,SUM("总金额") AS total_payment
    FROM example_table
    GROUP BY  "订单创建日期","收货地址 ") AS t1

'''
result = connection.execute(query).fetchall()
result[:20]


[('2020-02-01', '上海', 5855.0),
 ('2020-02-02', '上海', 6296.333333333333),
 ('2020-02-03', '上海', 9247.116666666667),
 ('2020-02-04', '上海', 9204.783333333333),
 ('2020-02-05', '上海', 8137.783333333333),
 ('2020-02-06', '上海', 5008.0),
 ('2020-02-07', '上海', 4962.666666666667),
 ('2020-02-09', '上海', 4688.0),
 ('2020-02-10', '上海', 3320.0),
 ('2020-02-11', '上海', 1099.3333333333333),
 ('2020-02-13', '上海', 67.0),
 ('2020-02-14', '上海', 2471.8233333333333),
 ('2020-02-17', '上海', 7250.646666666668),
 ('2020-02-18', '上海', 11132.52666666667),
 ('2020-02-19', '上海', 13485.703333333333),
 ('2020-02-20', '上海', 16202.313333333332),
 ('2020-02-21', '上海', 19493.533333333333),
 ('2020-02-22', '上海', 21403.966666666667),
 ('2020-02-23', '上海', 85819.23333333332),
 ('2020-02-24', '上海', 89782.89999999998)]

In [ ]:
query=""" 
SELECT
    address,
    order_date
FROM
    (SELECT
    "收货地址" AS address,
    "订单创建日期" AS order_date,
    min("订单创建日期")
    OVER(
        PARTITION BY "收货地址"
        ORDER BY "订单创建日期"  ASC
        ROWS
            BETWEEN
                1 PRECEDING
            AND CURRENT ROW) AS last_order_date
    FROM    example_table
    where "总金额">2000
        )
where last_order_date=order_date
"""
result = connection.execute(query).fetchall()
result

[('收货地址', '2020-02-02'),
 ('收货地址', '2020-02-18'),
 ('收货地址', '2020-02-23'),
 ('收货地址', '2020-02-24'),
 ('收货地址', '2020-02-28'),
 ('收货地址', '2020-02-29')]

: 

In [ ]:
# 对于订单数据
# 我们想要统计每个城市的首次有订单总金额 > 2000 的日期
query = '''
SELECT
    address
    ,order_date
FROM
    (SELECT
        "收货地址 " AS address
        ,"订单创建日期" AS order_date
        ,min("订单创建日期")                -- min() 聚合函数
        OVER(                             -- 定义 frame
            PARTITION BY "收货地址 "        -- frame 数据中只保留地址相同的
            ORDER BY "订单创建日期" ASC     -- 按照创建日期排序
            ROWS                          -- 按照表中的相对位置取数
                BETWEEN 
                    1 PRECEDING           -- 前一行数据
                AND CURRENT ROW
                ) AS last_order_date      -- 取出每条数据的“前一行”日期
    FROM example_table
    WHERE "总金额">2000)
WHERE last_order_date = order_date
'''
result = connection.execute(query).fetchall()
result

[('上海', '2020-02-04'),
 ('内蒙古自治区', '2020-02-26'),
 ('安徽省', '2020-02-28'),
 ('安徽省', '2020-02-28'),
 ('山东省', '2020-02-22'),
 ('广东省', '2020-02-09'),
 ('江苏省', '2020-02-18'),
 ('河北省', '2020-02-23'),
 ('河北省', '2020-02-23'),
 ('河南省', '2020-02-02'),
 ('重庆', '2020-02-29'),
 ('重庆', '2020-02-29'),
 ('陕西省', '2020-02-18')]

# GPT 答案

In [ ]:
# 查找 订单创建时间上首次出现“单笔交易总金额>5000最晚的城市/省份”

query="""   
SELECT 
    address,
    order_date
from 
(
    SELECT
    "收货地址 " as address,
    "订单创建日期" AS order_date,
    max("订单创建日期")
    over(
        PARTITION BY "收货地址 "
        order by "订单创建日期" DESC
        ROWS
        BETWEEN 
            CURRENT ROW
        AND 1 FOLLOWING
        )AS last_order_date
    FROM example_table
    where "总金额">5000
)
where last_order_date=order_date


"""
result=connection.execute(query).fetchall()
result

[('上海', '2020-02-24'), ('内蒙古自治区', '2020-02-26'), ('江苏省', '2020-02-18')]

In [ ]:
# 对于订单数据
# 我们想要统计每个城市的首次有订单总金额 > 2000 的日期
query = '''
SELECT
    address
    ,order_date
FROM
    (SELECT
        "收货地址 " AS address
        ,"订单创建日期" AS order_date
        ,min("订单创建日期")                -- min() 聚合函数
        OVER(                             -- 定义 frame
            PARTITION BY "收货地址 "        -- frame 数据中只保留地址相同的
            ORDER BY "订单创建日期" DESC     -- 按照创建日期排序
            ROWS                          -- 按照表中的相对位置取数
                BETWEEN 
                    1 PRECEDING           -- 前一行数据
                AND CURRENT ROW
                ) AS last_order_date      -- 取出每条数据的“前一行”日期
    FROM example_table
    WHERE "总金额">5000)
WHERE last_order_date = order_date
'''
result = connection.execute(query).fetchall()
result

[('上海', '2020-02-24'), ('内蒙古自治区', '2020-02-26'), ('江苏省', '2020-02-18')]

In [ ]:
query = """
-- 查询首次“单笔交易总金额 > 5000”的最晚城市/省份
WITH first_high_value_orders AS (
    SELECT 
        "收货地址 " AS address,
        MIN("订单创建日期") AS first_order_date
    FROM example_table
    WHERE "总金额" > 5000
    GROUP BY "收货地址 "
),
latest_city AS (
    SELECT 
        address,
        first_order_date
    FROM first_high_value_orders
    ORDER BY first_order_date DESC
    LIMIT 1
)
SELECT * 
FROM latest_city;
"""
result = connection.execute(query).fetchall()
result

[('内蒙古自治区', '2020-02-26')]

In [ ]:
# Q2：在订单数据集中，哪个城市/省份的订单金额（“总金额”字段）的方差最大？¶
  
query = """
WITH tmp AS (
    SELECT 
        "收货地址 " AS city_province,
        "总金额",
        AVG("总金额") OVER(PARTITION BY "收货地址 ") AS avg_amount
    FROM example_table
),
variance_calc AS (
    SELECT 
        city_province,
        SUM((total_amount - avg_amount) * (total_amount - avg_amount)) / COUNT(*) AS variance
    FROM (
        SELECT 
            city_province, 
            "总金额" AS total_amount, 
            avg_amount
        FROM tmp
    ) AS subquery
    GROUP BY city_province
)
SELECT city_province, variance
FROM variance_calc
ORDER BY variance DESC
LIMIT 1;
"""
result =connection.execute(query).fetchall()
result

[('上海', 10577678.603002107)]

In [ ]:
#Q3. 在订单数据集中，计算北京和上海的累计平均订单金额这一指标，两地这一指标差距最大的日期是几号？

query = """
WITH tmp AS (
    SELECT 
        "订单创建日期" AS order_date,
        "收货地址 " AS city_province,
        SUM("总金额") AS total_amount,
        COUNT(*) AS total_orders
    FROM example_table
    WHERE "收货地址 " IN ('北京', '上海')
    GROUP BY order_date, city_province
),
cumulative_data AS (
    SELECT 
        order_date,
        city_province,
        SUM(total_amount) OVER(PARTITION BY city_province ORDER BY order_date ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cum_price,
        SUM(total_orders) OVER(PARTITION BY city_province ORDER BY order_date ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cum_orders
    FROM tmp
),
cumulative_avg AS (
    SELECT 
        order_date,
        city_province,
        cum_price,
        cum_orders,
        cum_price * 1.0 / cum_orders AS cum_avg_order_amount
    FROM cumulative_data
),
difference AS (
    SELECT 
        a.order_date,
        ABS(a.cum_avg_order_amount - b.cum_avg_order_amount) AS diff
    FROM cumulative_avg a
    JOIN cumulative_avg b
    ON a.order_date = b.order_date AND a.city_province = '北京' AND b.city_province = '上海'
)
SELECT order_date, diff
FROM difference
ORDER BY diff DESC
LIMIT 1;
"""
result =connection.execute(query).fetchall()
result

[('2020-02-24', 87.65238560257924)]

In [ ]:
# Q4： 在 Q3 的数据中，北京有多少天的数据是高于上海的？¶
query = """
WITH tmp AS (
    SELECT 
        "订单创建日期" AS order_date,
        "收货地址 " AS city_province,
        SUM("总金额") AS total_amount,
        COUNT(*) AS total_orders
    FROM example_table
    WHERE "收货地址 " IN ('北京', '上海')
    GROUP BY order_date, city_province
),
cumulative_data AS (
    SELECT 
        order_date,
        city_province,
        SUM(total_amount) OVER(PARTITION BY city_province ORDER BY order_date ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cum_price,
        SUM(total_orders) OVER(PARTITION BY city_province ORDER BY order_date ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cum_orders
    FROM tmp
),
cumulative_avg AS (
    SELECT 
        order_date,
        city_province,
        cum_price,
        cum_orders,
        cum_price * 1.0 / cum_orders AS cum_avg_order_amount
    FROM cumulative_data
),
comparison AS (
    SELECT 
        a.order_date,
        a.cum_avg_order_amount AS beijing_avg,
        b.cum_avg_order_amount AS shanghai_avg
    FROM cumulative_avg a
    JOIN cumulative_avg b
    ON a.order_date = b.order_date AND a.city_province = '北京' AND b.city_province = '上海'
)
SELECT COUNT(*) AS days_beijing_higher
FROM comparison
WHERE beijing_avg > shanghai_avg;
"""
result =connection.execute(query).fetchall()
result

[(4,)]

In [ ]:
a1='内蒙古自治区'
a2='上海'
a3=24
a4=4

# 学习

In [ ]:
# 练习1： 写出上界为当前行向上取3，下界不设限制
# rows between 3 PRECEDING  and UNBOUNDED  FOLLOWING

#  小练习 2： 
# 尝试用自己的理解去解读 
# ORDER BY price, stock_number 
# ROWS BETWEEN UNBOUNDED PRECEDING AND 1 FOLLOWING 代表的数据范围



In [ ]:
#Q1：在订单数据集中，从订单创建 {时间} 上首次出现“单笔交易总金额>5000”最晚的城市/省份是？

query=""" 
WITH tmp as 
(
    SELECT 
    "收货地址 "  AS address,
    "订单创建时间" AS order_timestamp,
    max("订单创建时间") 
    over(
        PARTITION BY "收货地址 " 
        ORDER BY "订单创建时间"
        ROWS 
        BETWEEN
            CURRENT ROW 
        AND 
            1 FOLLOWING
        ) AS next_order_ts
    FROM example_table
    WHERE "总金额">5000
)

SELECT 
    order_timestamp,
    address
FROM
    tmp
WHERE
    next_order_ts=order_timestamp
ORDER BY 
    order_timestamp DESC
"""
result=connection.execute(query).fetchall()
result

[('2020-02-26 15:41:27', '内蒙古自治区'),
 ('2020-02-24 19:35:06', '上海'),
 ('2020-02-18 09:34:43', '江苏省')]

In [9]:
#### Q2：在订单数据集中，哪个城市/省份的订单金额（“总金额”字段）的方差最大？
query=""" 
WITH tmp AS 
(
    SELECT 
    "收货地址 " as address,
    "总金额"- avg("总金额") over(PARTITION BY "收货地址 ") as err
    FROM example_table
)
SELECT 
    address,
    avg(err*err)
FROM tmp
GROUP BY address
ORDER BY avg(err*err) DESC
"""
result=connection.execute(query).fetchall()
result


[('上海', 10577678.603002107),
 ('内蒙古自治区', 1186845.6629810715),
 ('江苏省', 73520.64513990036),
 ('重庆', 50238.765305920075),
 ('陕西省', 31238.220829015052),
 ('安徽省', 26926.05873402952),
 ('河南省', 20420.271303347014),
 ('河北省', 15775.931539268959),
 ('山东省', 14976.644869726226),
 ('广东省', 13685.630966898623),
 ('青海省', 12209.365540166205),
 ('天津', 11118.218922301376),
 ('吉林省', 10462.930384000141),
 ('北京', 10341.067078765202),
 ('新疆维吾尔自治区', 9329.56928953627),
 ('浙江省', 9003.615133954463),
 ('宁夏回族自治区', 8735.229115646254),
 ('贵州省', 8226.56439156817),
 ('江西省', 7766.078292249048),
 ('湖北省', 7447.872952888891),
 ('山西省', 7298.114323783125),
 ('甘肃省', 7079.228122399507),
 ('云南省', 7011.144334430783),
 ('湖南省', 6735.506953905839),
 ('四川省', 6278.123870071879),
 ('广西壮族自治区', 6009.2950354115965),
 ('海南省', 5465.206213369526),
 ('辽宁省', 4709.749603771099),
 ('黑龙江省', 4550.756033913717),
 ('福建省', 2711.7150120365964),
 ('西藏自治区', 682.7818666666667)]

In [11]:
#### Q3. 在订单数据集中，计算北京和上海的累计平均订单金额这一指标，两地这一指标差距最大的日期是几号？
query = '''
WITH tmp AS
(
SELECT 
    "收货地址 " AS address
    ,"订单创建日期" AS order_date
    ,SUM("总金额") AS price
    ,COUNT("总金额") AS orders
FROM example_table
WHERE "收货地址 " IN ('北京','上海')
GROUP BY "收货地址 ","订单创建日期"
)
,tmp1 AS
(
SELECT
    order_date
    ,SUM(price) OVER(PARTITION BY address ORDER BY order_date ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cum_price
    ,SUM(orders) OVER(PARTITION BY address ORDER BY order_date ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cum_orders
    ,address    
FROM tmp
)
,tmp2 AS
(
SELECT
    order_date
    /*
    思路：
    因为不需要知道是个城市，所以只需要对于 order_date 分组，得出每组下的差值（值 - 最小值 >= 0）
    */
    ,cum_price/cum_orders - min(cum_price/cum_orders) OVER(PARTITION BY order_date) AS margin
FROM tmp1
)
SELECT order_date
        ,margin
FROM tmp2
ORDER BY margin DESC
'''
result = connection.execute(query).fetchall()
result

[('2020-02-24', 87.65238560257924),
 ('2020-02-25', 74.52541065913273),
 ('2020-02-26', 64.39660004168617),
 ('2020-02-27', 60.138257495841316),
 ('2020-02-28', 53.690752194786185),
 ('2020-02-29', 50.02295812786353),
 ('2020-02-02', 32.110602593440134),
 ('2020-02-01', 28.02916666666667),
 ('2020-02-03', 25.438804286520025),
 ('2020-02-04', 22.365657421674385),
 ('2020-02-05', 16.073480316579307),
 ('2020-02-07', 14.248953648036817),
 ('2020-02-06', 11.669101196033964),
 ('2020-02-21', 9.977065490689725),
 ('2020-02-11', 9.35837063553491),
 ('2020-02-10', 9.010339252962567),
 ('2020-02-23', 8.87058515561742),
 ('2020-02-17', 7.2659448864102245),
 ('2020-02-22', 7.240816274724807),
 ('2020-02-18', 5.6922594618032605),
 ('2020-02-20', 4.803068499181165),
 ('2020-02-09', 3.533039880381679),
 ('2020-02-19', 2.08935529541489),
 ('2020-02-01', 0.0),
 ('2020-02-02', 0.0),
 ('2020-02-03', 0.0),
 ('2020-02-04', 0.0),
 ('2020-02-05', 0.0),
 ('2020-02-06', 0.0),
 ('2020-02-07', 0.0),
 ('2020-02-

In [ ]:
### Q4： 在 Q3 的数据中，北京有多少天的数据是高于上海的？
query = '''
WITH tmp AS
(
SELECT 
    "收货地址 " AS address
    ,"订单创建日期" AS order_date
    ,SUM("总金额") AS price
    ,COUNT("总金额") AS orders
FROM example_table
WHERE "收货地址 " IN ('北京','上海')
GROUP BY "收货地址 ","订单创建日期"
)
,tmp1 AS
(
SELECT
    order_date
    /*
    按照 address 字段计算 累积总金额 （cum_price） 和 累积总订单数 (cum_orders)
    */
    ,SUM(price) OVER(PARTITION BY address ORDER BY order_date ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cum_price
    ,SUM(orders) OVER(PARTITION BY address ORDER BY order_date ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cum_orders
    ,address    
FROM tmp
)
,tmp2 AS
(
SELECT
    order_date
    ,address
    /*
    代码设计思路：
    1. cum_price/cum_orders 是我们要计算的指标，下面简称【指标 A】
    2. min(cum_price/cum_orders)... ： 我们在两个城市当中选择该指标较小的值，如果这个值和当前行的值是一样的，说明这一行是“小”的值
       ,反之，如果两个值不一样，说明当前行的值时两个比较值中较大的
    3. FILTER(WHERE address<>"北京") 并不是必须的，其主要目的是为了让输出的结果可以更好的被引用（可以去掉这个 FILTER 看看效果）
    4. 在 min 聚合函数当中只保留 “上海” 的数据，意味着只要 北京>上海，则输出结果一定为 > 0 【且】 “上海”的结果一定为 0
    5. 这样我们可以通过 a. 筛选地址 b.筛选数值 的形式得出我们要的结果
    6. 同时，也可以直接通过排序 （ORDER BY） 来获得结果，不使用 FILTER 的话，上海 > 北京 部分的数据会扰乱排序
    */
    ,cum_price/cum_orders - min(cum_price/cum_orders) FILTER(WHERE address <> "北京") OVER(PARTITION BY order_date) AS bj_is_large
FROM tmp1
ORDER BY cum_price/cum_orders - min(cum_price/cum_orders) FILTER(WHERE address <> "北京") OVER(PARTITION BY order_date) DESC
)
SELECT *
FROM tmp2
-- WHERE bj_is_large>0
'''
result = connection.execute(query).fetchall()
result

[('2020-02-21', '北京', 9.977065490689725),
 ('2020-02-23', '北京', 8.87058515561742),
 ('2020-02-22', '北京', 7.240816274724807),
 ('2020-02-20', '北京', 4.803068499181165),
 ('2020-02-01', '上海', 0.0),
 ('2020-02-02', '上海', 0.0),
 ('2020-02-03', '上海', 0.0),
 ('2020-02-04', '上海', 0.0),
 ('2020-02-05', '上海', 0.0),
 ('2020-02-06', '上海', 0.0),
 ('2020-02-07', '上海', 0.0),
 ('2020-02-09', '上海', 0.0),
 ('2020-02-10', '上海', 0.0),
 ('2020-02-11', '上海', 0.0),
 ('2020-02-13', '上海', 0.0),
 ('2020-02-14', '上海', 0.0),
 ('2020-02-17', '上海', 0.0),
 ('2020-02-18', '上海', 0.0),
 ('2020-02-19', '上海', 0.0),
 ('2020-02-20', '上海', 0.0),
 ('2020-02-21', '上海', 0.0),
 ('2020-02-22', '上海', 0.0),
 ('2020-02-23', '上海', 0.0),
 ('2020-02-24', '上海', 0.0),
 ('2020-02-25', '上海', 0.0),
 ('2020-02-26', '上海', 0.0),
 ('2020-02-27', '上海', 0.0),
 ('2020-02-28', '上海', 0.0),
 ('2020-02-29', '上海', 0.0),
 ('2020-02-19', '北京', -2.08935529541489),
 ('2020-02-09', '北京', -3.533039880381679),
 ('2020-02-18', '北京', -5.6922594618032605),
 ('2

: 